In [1]:
import re
import urllib
import urllib.request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
%matplotlib inline
plt.style.use('ggplot')
sns.set_style('darkgrid')

In [2]:
# embaralhei a lista de players pra nao pegar só de GM - NAO PRECISA RODAR ESSA CEDULA DNV
# shuffled_players.to_csv('shuffled_player_list_chesscom.csv',index = False)

In [3]:
players = pd.read_csv('shuffled_player_list_chesscom.csv')

In [4]:
players

,username,title,bullet_rating,blitz_rating,rapid_rating
0,michaelq2d5,GM,2793,2765,2903
1,bernd_stromberg,CM,2619,2701,0
2,g2_coder,FM,2350,2200,0
3,mennabarreto,FM,2251,2407,0
4,radojeiriz,FM,2388,2271,2489
...,...,...,...,...,...
8556,skiiier,FM,2492,2477,0
8557,volodymyrmolyboha,NM,2431,2503,2189
8558,miodrag-jevtic,FM,2378,2356,1936
8559,jerryle,NM,2214,2251,1365


In [5]:
# resolve erro "This event loop is already running"
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [6]:
# instala e importa https://github.com/sarartur/chess.com
# wrapper para dados publicados pelo chess.com
!pip install chessdotcom
import chessdotcom

ERROR: Could not find a version that satisfies the requirement chessdotcom
ERROR: No matching distribution found for chessdotcom


In [7]:
# peguei um role do stack overflow e editei, pega os anos/meses que teve jogo de cada pessoa

def getDates(user):
    username = user #change 
    baseUrl = "https://api.chess.com/pub/player/" + username + "/games/"
    archivesUrl = baseUrl + "archives"

    #read the archives url and store in a list
    f = urllib.request.urlopen(archivesUrl)
    archives = f.read().decode("utf-8")
    archives = archives.replace("{\"archives\":[\"", "\",\"")
    archivesList = archives.split("\",\"" + baseUrl)
    archivesList[len(archivesList)-1] = archivesList[len(archivesList)-1].rstrip("\"]}")
    #print(archivesList)
    month = []
    year = []
    # trata caso em que o jogador nao tem jogos
    if archivesList[0] == '{"archives":[':
        print('Erro - Jogador não possui partidas registradas')
        return(month, year) 
    archivesList.remove('')
    for i in archivesList:
        year.append(int(i[0:4]))
        month.append(int(i[5:]))
    return(month, year)

In [8]:
games_db = pd.read_csv('empty_games.csv')
games_db.head()

,id,ranked,time_control,status,winner,white,black,moves


In [9]:
def checkstatus(white_result, black_result):
    if white_result != 'win':
        return white_result
    else:
        return black_result
    
def checkwinner(white, black):
    if white['result'] == 'win':
        return white['username']
    elif black['result'] == 'win':
        return black['username']
    else:
        return 'draw'

def trataPGN(pgn):
    # remove clock
    txt = '{\[%clk \d+:\d+:\d+(\.\d+)?\]}'
    a = re.sub(txt, "", pgn)

    # remove header
    txt = '\[.+\]'
    a = re.sub(txt, "", a)
    a

    # remove \n
    txt = '\\n'
    a = re.sub(txt, "", a)
    a
    
    return(a)    

In [20]:
# escolhe que jogadores voce quer pegar
players_to_db = players[55:65]
players_to_db

,username,title,bullet_rating,blitz_rating,rapid_rating
55,atalik,GM,2623,2660,2520
56,veneteanuc,NM,2183,2190,0
57,ebrimabah,FM,0,2162,1954
58,busygin,NM,2202,1909,2043
59,jackvino,FM,2514,2249,1952
60,chessforthesun,NM,2587,2609,2273
61,keitharkell,GM,2361,2477,2408
62,911master,NM,0,1900,1872
63,ruzigura,CM,1641,1628,1777
64,cristiancamilosoto,FM,2267,2254,0


In [21]:
games_db = pd.read_csv('empty_games.csv')

for k in players_to_db['username']:
    month, year = getDates(k)
    print(k)
    for i in range(len(month)):
        response = chessdotcom.get_player_games_by_month(k,month=month[i],year=year[i]).json
        for j in (response['games']):
            try:
                temp = {
                    'id':j['uuid'],
                    'ranked':j['rated'],
                    'time_control': j['time_class'],
                    'status': checkstatus(j['white']['result'],j['black']['result']),
                    'winner': checkwinner(j['white'],j['black']),
                    'white': (j['white']['username']),
                    'black': (j['black']['username']),
                    'moves': trataPGN(j['pgn'])
                }
                #print((j['white']['result'],j['black']['result']))
                games_db = games_db.append(temp,ignore_index=True)
            except:
                temp = 0

atalik
veneteanuc
ebrimabah
busygin
jackvino
chessforthesun
keitharkell
911master
ruzigura
cristiancamilosoto


In [22]:
games_db

,id,ranked,time_control,status,winner,white,black,moves
0,e6cd2f50-bb4f-11dd-8000-000000010001,True,blitz,checkmated,Atalik,Atalik,ubahn,1. d4 1... e6 2. c4 2... d6 3. e4 3... c6...
1,a7564b64-bdba-11dd-8000-000000010001,True,bullet,resigned,Atalik,martinZH,Atalik,1. d4 1... Nf6 2. c4 2... e6 3. Nc3 3... ...
2,f603e53c-bdba-11dd-8000-000000010001,True,blitz,timeout,Atalik,Atalik,rock_baby_rock,1. d4 1... Nf6 2. c4 2... e6 3. Nc3 3... ...
3,893d1b0c-bdbb-11dd-8000-000000010001,True,blitz,timeout,okahais,okahais,Atalik,1. e4 1... e6 2. d4 2... d5 3. Nd2 3... N...
4,b8f12078-bdc5-11dd-8000-000000010001,True,blitz,resigned,Atalik,Atalik,mattkvagabond,1. d4 1... c5 2. d5 2... d6 3. e4 3... e6...
...,...,...,...,...,...,...,...,...
63780,8d7277fc-9309-11eb-b848-536974010001,True,bullet,checkmated,cristiancamilosoto,cristiancamilosoto,LordIron,1. c4 1... b6 2. Nc3 2... Bb7 3. e4 3... ...
63781,f889ef74-9309-11eb-b848-536974010001,True,bullet,checkmated,cristiancamilosoto,cristiancamilosoto,allvim,1. c4 1... Nf6 2. Nc3 2... g6 3. e4 3... ...
63782,5e8bacf5-930a-11eb-b848-536974010001,True,bullet,checkmated,kinga2008,kinga2008,cristiancamilosoto,1. e4 1... c5 2. Nf3 2... e6 3. c3 3... N...
63783,c644683c-930a-11eb-b848-536974010001,True,bullet,timeout,Manukyan_Artak,cristiancamilosoto,Manukyan_Artak,1. c4 1... c6 2. Nc3 2... Nf6 3. e4 3... ...


In [23]:
# salva em csv
games_db.to_csv('games_db_55_64.csv',index = False)

In [ ]:
# Pesquisa de nomes no dataframe
#w = 'chesspartner123'
#texts = [*filter(lambda x: w in x, games_db['winner'])]
#print(*texts, sep=f'\n\n{"=" * 200}\n\n')